## Import libraries 

In [16]:
import pandas as pd
import numpy as np

## Convert BonBanh data and Chotot data to same format

#### Read data from file csv

In [17]:
bonbanh = pd.read_csv('Data/BonBanh.csv')
bonbanh.head()

,Tên xe,Năm sản xuất,Tình trạng,Số km đã đi,Xuất xứ,Kiểu dáng,Hộp số,Động cơ,Màu ngoại thất,Màu nội thất,Dẫn động,Số chỗ ngồi,Giá xe
0,Xe Peugeot 2008 Active 1.2 AT,2022,Xe đã dùng,"30,000 Km",Lắp ráp trong nước,Crossover,Số tự động,Xăng 1.2 L,Đỏ,Đen,FWD - Dẫn động cầu trước,5 chỗ,679 Triệu
1,Xe Mercedes Benz C class C300 AMG,2014,Xe đã dùng,"90,000 Km",Lắp ráp trong nước,Sedan,Số tự động,Xăng 3.0 L,Trắng,Đen,RFD - Dẫn động cầu sau,5 chỗ,499 Triệu
2,Xe Mercedes Benz E class E200 Exclusive,2022,Xe đã dùng,500 Km,Lắp ráp trong nước,Sedan,Số tự động,Xăng 2.0 L,Đen,Nâu,RFD - Dẫn động cầu sau,5 chỗ,2 Tỷ 229 Triệu
3,Xe Mercedes Benz C class C200 Avantgarde,2021,Xe đã dùng,80 Km,Lắp ráp trong nước,Sedan,Số tự động,Xăng 1.5 L,Xanh,Nâu,RFD - Dẫn động cầu sau,5 chỗ,1 Tỷ 399 Triệu
4,Xe Mercedes Benz E class E200 Exclusive,2022,Xe đã dùng,"8,888 Km",Lắp ráp trong nước,Sedan,Số tự động,Xăng 2.0 L,Trắng,Nâu,RFD - Dẫn động cầu sau,5 chỗ,1 Tỷ 788 Triệu


In [18]:
chotot = pd.read_csv('Data/Chotot.csv')
chotot.head()

,Tiêu đề,Hãng,Dòng xe,Năm sản xuất,Số km đã đi,Tình trạng,Hộp số,Nhiên liệu,Xuất xứ,Kiểu dáng,Số chỗ,Giá
0,bán xe Xpander chính chủ,Mitsubishi,Xpander,2019,50000.0,Đã sử dụng,Tự động,Xăng,Nước khác,NaN,NaN,460.000.000 đ
1,Lên đời bán xe Pajero Sport mua hãng còn mới,Mitsubishi,Pajero Sport,2021,60000.0,Đã sử dụng,Tự động,Dầu,Thái Lan,NaN,NaN,830.000.000 đ
2,"Ford Everest Wildtrak, Đủ màu giao ngay",Ford,Everest,2023,NaN,Mới,Bán tự động,Dầu,Thái Lan,SUV / Cross over,NaN,1.409.000.000 đ
3,Everest 4x4 2023 giảm giá tới 120tr góp từ 250tr,Ford,Everest,2023,NaN,Mới,Tự động,Dầu,Thái Lan,SUV / Cross over,NaN,1.352.000.000 đ
4,cần bán xe lux A 2.0 đời 2019 Bản prelium mới đét,Vinfast,Lux A2.0,2019,26000.0,Đã sử dụng,Tự động,Xăng,Việt Nam,NaN,NaN,605.000.000 đ


#### Change value '-' into null

In [19]:
for col in bonbanh:
    bonbanh[col] = bonbanh[col].replace('-', np.nan)

#### Remove all price-missing car

In [20]:
chotot = chotot.dropna(subset=['Giá'])

#### Convert **Number of km traveled** to int and change it to *0* for all new cars

In [21]:
bonbanh['Số km đã đi'] = bonbanh['Số km đã đi'].replace('Xe mới', '0 Km')
bonbanh['Số km đã đi'] = bonbanh['Số km đã đi'].str.replace('Km', '')
def change_to_int(so_km):
    km = 0
    so_km = [int(x) for x in so_km.split(',')]
    n = len(so_km)
    for i in range(n):
        km += 1000**(n-i-1)*so_km[i] 
    return km
bonbanh['Số km đã đi'] = bonbanh['Số km đã đi'].apply(change_to_int)

In [22]:
chotot.loc[chotot['Tình trạng'] == 'Mới', 'Số km đã đi'] = 0

#### Convert **Price** into *Million* unit

In [23]:
def convert_price(price):
    price = price.split(" ")
    if len(price) == 4:
        return int(price[0])*1000 + int(price[2])
    else:
        if price[-1] == 'Tỷ':
            return int(price[0]) * 1000
        else:
            return int(price[0])
bonbanh['Giá xe'] = bonbanh['Giá xe'].apply(convert_price)
bonbanh = bonbanh.rename(columns = {'Giá xe': 'Giá (triệu)'})
bonbanh['Giá (triệu)'] = bonbanh['Giá (triệu)'].astype('int64')

In [24]:
chotot['Giá'] = chotot['Giá'].astype(str).str.rstrip('đ')
def price_convert(x):
    l = x.split('.')
    n = ''.join(l[0:len(x)-2:1])
    return n[0:-7]
chotot['Giá'] = chotot['Giá'].apply(price_convert)
chotot = chotot.rename(columns = {'Giá': 'Giá (triệu)'})
chotot['Giá (triệu)'] = chotot['Giá (triệu)'].astype('int64')

#### Change **State**

In [25]:
chotot['Tình trạng'] = chotot['Tình trạng'].replace('Đã sử dụng', 'Xe đã dùng')
chotot['Tình trạng'] = chotot['Tình trạng'].replace('Mới', 'Xe mới')

#### Change **Gearbox**

In [26]:
def change_gearbox(x):   
    if x == 'Tự động' or x == '4':
        return 'Số tự động'
    elif x == 'Bán tự động':
        return 'Số hỗn hợp'
    elif x == 'Số sàn' or x == '5':
        return 'Số tay'
    else:
        return x
chotot['Hộp số'] = chotot['Hộp số'].apply(change_gearbox)

#### Change **Design** 

In [27]:
def change_design(x):
    if x == 'Bán tải / Pickup' or x == 'Pick-up (bán tải)':
        return 'Pickup'
    elif x == 'SUV' or x == 'Crossover':
        return 'SUV /Crossover'
    elif x == 'Van' or x == 'Minivan (MPV)':
        return 'Van/Minivan'
    elif x == 'Coupe (2 cửa)':
        return 'Coupe'
    elif x == 'Convertible/Cabriolet' or x == 'Mui trần':
        return 'Convertible'
    elif x == 'Truck' or x == 'Wagon':
        return 'Kiểu dáng khác'
    else:
        return x
bonbanh['Kiểu dáng'] = bonbanh['Kiểu dáng'].apply(change_design)
chotot['Kiểu dáng'] = chotot['Kiểu dáng'].apply(change_design)

#### Change **Origin** into *Domestical* or *Imported* 

In [28]:
def change_origin(x):
    if x == 'Việt Nam' or x == 'Lắp ráp trong nước':
        return 'Trong nước'
    elif x in ['Nước khác', 'Thái Lan', 'Hàn Quốc', 'Đang cập nhật', 'Mỹ', 'Nhật Bản', 'Đức', 'Đài Loan', 'Ấn Độ', 'Trung Quốc']:
        return 'Nhập khẩu'
    else:
        return x
bonbanh['Xuất xứ'] = bonbanh['Xuất xứ'].apply(change_origin)
chotot['Xuất xứ'] = chotot['Xuất xứ'].apply(change_origin)

#### Take **Capacity** and **Engine** from **Fuel type**

In [29]:
bonbanh['Dung tích'] = bonbanh['Động cơ'].str.extract(r'(\d\.\d)')
bonbanh = bonbanh.rename(columns = {'Động cơ': 'Nhiên liệu'})
bonbanh['Nhiên liệu'] = bonbanh['Nhiên liệu'].str.extract(r'(Xăng|Dầu|Hybrid|Điện)')
bonbanh.loc[bonbanh['Nhiên liệu'] == 'Điện', 'Dung tích'] = 0.0
bonbanh['Dung tích'] = bonbanh['Dung tích'].astype('float64')

In [30]:
chotot['Dung tích'] = np.nan
chotot.loc[chotot['Nhiên liệu'] == 'Điện', 'Dung tích'] = 0.0

#### Take **Brand** and **Series** from **Name**

In [31]:
bonbanh['Tên xe'] = bonbanh['Tên xe'].str.replace('Xe ', '')
check =  lambda x : 'Nan'.join(x.split()[:1]) if x.split()[0] != "Mercedes" else ' '.join(x.split()[:2])
cartype = bonbanh['Tên xe'].apply(check)
bonbanh['Hãng'] = cartype

bonbanh['Tên xe'] = bonbanh['Tên xe'].apply(lambda x : ' '.join(x.split()[1:]) if x.split()[0] != "Mercedes" else ' '.join(x.split()[2:]))

classcheck = lambda x: np.nan if x.split() == [] else (' '.join(x.split()[:2]) if (x.split()[0] == "4" or x.split()[0]=="Range Rover" or x.split()[0]=="Lux" or x.split()[0]== "Range" or x.split()[0] == "Corolla" ) else ' '.join(x.split()[:1]))
bonbanh['Dòng xe'] = bonbanh['Tên xe'].apply(classcheck)

def cross_toyota_check(x):
    if x["Tên xe"].split().count("Cross") > 0:
        return x["Dòng xe"]+ " " + "Cross"
    elif x["Tên xe"].split().count("Altis") > 0:
        return x["Dòng xe"]+ " " + "Altis"
    else:
        return x["Dòng xe"]

bonbanh["Dòng xe"] = bonbanh.apply(cross_toyota_check, axis=1)

bonbanh['Dòng xe'] = bonbanh['Dòng xe'].str.replace(' ', '')
bonbanh['Dòng xe'] = bonbanh['Dòng xe'].str.replace('-', '')

bonbanh['Dòng xe'] = bonbanh['Dòng xe'].str.lower()

def mercedes_class(x):
    if x["Hãng"] == "Mercedes":
        if x['Dòng xe'] == "c" or x['Dòng xe'] == "e" or x['Dòng xe'] == "s" or x['Dòng xe'] == "g" or x['Dòng xe'] == "a" or x['Dòng xe'] == "b":
            return x["Dòng xe"] + "class"
        else:
            return x["Dòng xe"]
    else:
        return x["Dòng xe"]
    
bonbanh["Dòng xe"] = bonbanh.apply(mercedes_class, axis=1)

In [32]:
def corolla_altis(x):
    if x["Dòng xe"] == "Corolla":
        return x["Dòng xe"] + " " + "altis"
    else:
        return x["Dòng xe"]
chotot["Dòng xe"] = chotot.apply(corolla_altis, axis=1)

In [33]:
chotot["Dòng xe"] = chotot["Dòng xe"].str.lower()

#### Convert **Number of seats** to int

In [34]:
bonbanh['Số chỗ ngồi'] = bonbanh['Số chỗ ngồi'].str.replace('chỗ', '')
bonbanh = bonbanh.rename(columns = {'Số chỗ ngồi': 'Số chỗ'})

In [35]:
chotot = chotot[chotot['Số chỗ'] != 'Khác']

In [36]:
bonbanh['Số chỗ'] = bonbanh['Số chỗ'].astype('float64')
chotot['Số chỗ'] = chotot['Số chỗ'].astype('float64')

#### Select same columns in the same order

In [37]:
bonbanh = bonbanh[['Hãng','Dòng xe', 'Năm sản xuất','Tình trạng', 'Số km đã đi', 'Xuất xứ', 'Kiểu dáng', 'Hộp số', 'Nhiên liệu', 'Dung tích', 'Số chỗ', 'Giá (triệu)']]
chotot  =  chotot[['Hãng','Dòng xe', 'Năm sản xuất','Tình trạng', 'Số km đã đi', 'Xuất xứ', 'Kiểu dáng', 'Hộp số', 'Nhiên liệu', 'Dung tích', 'Số chỗ', 'Giá (triệu)']]

## Handle missing value

#### Fill missing values in **Design**, **Number of seats**, **Capacity** column

In [38]:
def find_car(brand,series):
    return bonbanh[(bonbanh['Hãng'] == brand) & (bonbanh["Dòng xe"] == series)]

def fill_design(x):
    data1 = find_car(x['Hãng'],x['Dòng xe'])
    if data1.empty:
        return x['Kiểu dáng']
    else:
        return data1['Kiểu dáng'].iloc[0]

def fill_seat(x):
    data1 = find_car(x['Hãng'],x['Dòng xe'])
    if data1.empty:
        return x['Số chỗ']
    else:
        return data1['Số chỗ'].iloc[0]

def fill_capacity(x):
    data1 = find_car(x['Hãng'],x['Dòng xe'])
    if data1.empty:
        return x['Dung tích']
    else:
        return data1['Dung tích'].iloc[0]

chotot["Kiểu dáng"] = chotot.apply(fill_design,axis=1)
chotot["Số chỗ"] = chotot.apply(fill_seat,axis=1)
chotot["Dung tích"] = chotot.apply(fill_capacity,axis=1)

## Combine **bonbanh** and **chotot**

In [39]:
data = pd.concat([bonbanh, chotot], ignore_index=True)

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38351 entries, 0 to 38350
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Hãng          38350 non-null  object 
 1   Dòng xe       38342 non-null  object 
 2   Năm sản xuất  38350 non-null  object 
 3   Tình trạng    38351 non-null  object 
 4   Số km đã đi   38088 non-null  float64
 5   Xuất xứ       34487 non-null  object 
 6   Kiểu dáng     34565 non-null  object 
 7   Hộp số        38342 non-null  object 
 8   Nhiên liệu    38343 non-null  object 
 9   Dung tích     32874 non-null  float64
 10  Số chỗ        33203 non-null  float64
 11  Giá (triệu)   38351 non-null  int64  
dtypes: float64(3), int64(1), object(8)
memory usage: 3.5+ MB


In [41]:
data = data.drop('Dòng xe', axis=1)

In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38351 entries, 0 to 38350
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Hãng          38350 non-null  object 
 1   Năm sản xuất  38350 non-null  object 
 2   Tình trạng    38351 non-null  object 
 3   Số km đã đi   38088 non-null  float64
 4   Xuất xứ       34487 non-null  object 
 5   Kiểu dáng     34565 non-null  object 
 6   Hộp số        38342 non-null  object 
 7   Nhiên liệu    38343 non-null  object 
 8   Dung tích     32874 non-null  float64
 9   Số chỗ        33203 non-null  float64
 10  Giá (triệu)   38351 non-null  int64  
dtypes: float64(3), int64(1), object(7)
memory usage: 3.2+ MB


In [43]:
data.to_csv('Data/data.csv', index=False)